In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/santa-2023/sample_submission.csv
/kaggle/input/santa-2023/puzzles.csv
/kaggle/input/santa-2023/puzzle_info.csv


In [ ]:
import numpy as np
import random
import pandas as pd
import gc  # Garbage Collector interface
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

# DQN Agent Class
class DQNAgent:
    def __init__(self, state_size, action_size, memory_size=1000, gamma=0.95, epsilon=1.0, epsilon_min=0.01, epsilon_decay=0.995):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = []  # Using list instead of deque to control memory size directly
        self.memory_size = memory_size
        self.gamma = gamma
        self.epsilon = epsilon
        self.epsilon_min = epsilon_min
        self.epsilon_decay = epsilon_decay
        self.model = self._build_model()

    def _build_model(self):
        model = Sequential()
        model.add(Dense(16, input_dim=self.state_size, activation='relu', dtype='float32'))  # Smaller layer
        model.add(Dense(16, activation='relu', dtype='float32'))  # Smaller layer
        model.add(Dense(self.action_size, activation='linear', dtype='float32'))
        model.compile(loss='mse', optimizer=Adam(learning_rate=0.001))
        return model

    def remember(self, state, action, reward, next_state, done):
        if len(self.memory) < self.memory_size:
            self.memory.append((state, action, reward, next_state, done))
        else:
            self.memory.pop(0)  # Remove the oldest experience if the memory is full
            self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, min(len(self.memory), batch_size))
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0, batch_size=batch_size)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

# Puzzle Data Processing Functions
def determine_state_size(puzzles_df, puzzle_category):
    filtered_puzzles = puzzles_df[puzzles_df['puzzle_type'].str.startswith(puzzle_category)]
    if not filtered_puzzles.empty:
        example_puzzle = filtered_puzzles.iloc[0]
        state_size = len(example_puzzle['solution_state'].split(';'))
        return state_size
    else:
        return None

def determine_action_size(puzzle_info_df, puzzle_category):
    filtered_info = puzzle_info_df[puzzle_info_df['puzzle_type'].str.startswith(puzzle_category)]
    if not filtered_info.empty:
        allowed_moves = eval(filtered_info.iloc[0]['allowed_moves'])
        action_size = len(allowed_moves)
        return action_size
    else:
        return None

# Placeholder for environment interaction
def interact_with_environment(agent, state, puzzle_category):
    # Replace this with the actual interaction with the environment
    next_state = state  # Example placeholder for the next state
    reward = 1  # Example placeholder for reward
    done = True  # Example placeholder for completion status
    return next_state, reward, done

# Training Loop for Each Puzzle Type
def train_agent_for_puzzle_type(agent, puzzle_category, puzzles_df, episodes=1000, batch_size=32):
    print(f"Starting training for {puzzle_category} puzzles.")
    for episode in range(episodes):
        state = np.random.rand(1, agent.state_size)  # Replace with actual initial state
        done = False
        while not done:
            action = agent.act(state)
            next_state, reward, done = interact_with_environment(agent, state, puzzle_category)
            agent.remember(state, action, reward, next_state, done)
            state = next_state
        agent.replay(batch_size)
    print(f"Training completed for {puzzle_category} puzzles.")

# Function to train and save an agent
def train_and_save_agent(agent, puzzle_category, puzzles_df):
    train_agent_for_puzzle_type(agent, puzzle_category, puzzles_df, episodes=500, batch_size=16)  # Reduced episodes and batch size
    agent.model.save(f"{puzzle_category}_agent_model.h5")
    del agent
    gc.collect()

# Main Execution Flow
if __name__ == "__main__":
    puzzles_df = pd.read_csv("/kaggle/input/santa-2023/puzzles.csv")
    puzzle_info_df = pd.read_csv("/kaggle/input/santa-2023/puzzle_info.csv")

    puzzle_categories = ['cube', 'wreath', 'globe']
    for puzzle_category in puzzle_categories:
        state_size = determine_state_size(puzzles_df, puzzle_category)
        action_size = determine_action_size(puzzle_info_df, puzzle_category)

        if state_size is not None and action_size is not None:
            print(f"Creating and training agent for {puzzle_category} puzzles.")
            agent = DQNAgent(state_size, action_size)
            train_and_save_agent(agent, puzzle_category, puzzles_df)
            # Clear memory after training each agent
            del agent
            gc.collect()


Creating and training agent for cube puzzles.
Starting training for cube puzzles.
1/1 [==============================] - ETA: 0s

In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model

# Assuming DQNAgent and other necessary functions are defined above this part

# Function to load a trained DQN agent from a saved file
def load_agent_model(file_path, state_size, action_size):
    agent = DQNAgent(state_size, action_size)
    agent.model = load_model(file_path)
    return agent

# Functions for generating solutions (generate_solution_for_puzzle, etc.)

# Main script for generating submission data
if __name__ == "__main__":
    # Load puzzle data
    puzzles_df = pd.read_csv("/kaggle/input/santa-2023/puzzles.csv")
    puzzle_info_df = pd.read_csv("/kaggle/input/santa-2023/puzzle_info.csv")

    # Load trained agents
    agents = {}
    for puzzle_category in ['cube', 'wreath', 'globe']:
        state_size = determine_state_size(puzzles_df, puzzle_category)
        action_size = determine_action_size(puzzle_info_df, puzzle_category)
        if state_size is not None and action_size is not None:
            agent_model_file = f"/kaggle/working/{puzzle_category}_agent_model.h5"
            agents[puzzle_category] = load_agent_model(agent_model_file, state_size, action_size)

    # Generate submission data
    submission_data = []
    for index, row in puzzles_df.iterrows():
        puzzle_id = row['id']
        puzzle_type = row['puzzle_type'].split('_')[0]  # Extract base puzzle type
        initial_state = row['initial_state']  # Adjust if state needs formatting

        agent = agents.get(puzzle_type)
        if agent is not None:
            solution_moves = generate_solution_for_puzzle(agent, initial_state, puzzle_type)
            submission_data.append({'id': puzzle_id, 'moves': solution_moves})
        else:
            print(f"No agent found for puzzle type: {puzzle_type}")

    # Convert to DataFrame and Save
    submission_df = pd.DataFrame(submission_data)
    submission_df.to_csv('submission.csv', index=False)
